In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class PosyLayer(nn.Module):
    
    def __init__(self,len_x, M):
        
        super(PosyLayer, self).__init__()
        
        self.len_x = len_x
        
        self.M = M
        
        self.w = nn.Parameter(6*torch.rand(self.len_x, self.M)-3)
    
    def forward(self, x):
        
        y = torch.log(x)
        
        z = [torch.log(1+torch.exp(torch.matmul(self.w[:,i], y))) for i in range(self.M)]
        
        monomial = torch.stack([torch.exp(zz)-1 for zz in z])
        
        return monomial #which is of length self.M
        

In [3]:
class PosyNet(nn.Module):

    def __init__(self):
        super(PosyNet, self).__init__()
        
        # input: 5-vector (state of rocket: x, z, vx, vz, m)
        # output: 2-vector (thrust: mag, direction)
        
        # hidden: 3-posynomial layer
        
        self.posy = PosyLayer(len_x = 5, M = 3)
        
        # an affine operation: y = Wx + b
        #self.fc = nn.Linear(3, 2)  

    def forward(self, x):
        
        m = self.posy(x)
        
        #out = self.fc(monomials)
        
        out1 = 2*m[0] - (12/9)*m[1]
        out2 = (2-4/3)*m[2]
        
        return torch.stack([out1, out2])

In [50]:
def compute_actual(state):
    
    x, z, vx, vz, m = state
    
    tgo = -3*z/vz
    
    ax = -6*vx/tgo - 12*x/tgo**2
    
    az = -6*vz/tgo - 12*z/tgo**2
    
    return torch.stack([ax, m*az])
    

In [51]:
net = PosyNet()
print(net)

print(list(net.parameters()))

PosyNet(
  (posy): PosyLayer()
)
[Parameter containing:
tensor([[ 1.778,  1.529, -0.505],
        [-2.890, -0.394, -1.673],
        [-0.253, -1.269, -1.228],
        [-2.545,  1.282, -2.293],
        [-1.335, -2.077, -2.322]], requires_grad=True)]


In [65]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=5e-9, momentum=0)

torch.set_printoptions(precision=3, sci_mode=False)

In [66]:
loss_list = [0,0,0,0,0]
torch.norm(loss_list)

AttributeError: 'list' object has no attribute 'dim'

In [69]:
running_loss = 0
for epoch in range(100):

    state = 0.2+torch.rand(5)
    actual = compute_actual(state)

    optimizer.zero_grad()
    net_prediction = net(state)

    #loss = 0.5*sum(torch.log(net_prediction**2) - torch.log(actual**2)) + 0.001*torch.norm(net.posy.w, p=1)
    loss = criterion(actual, net_prediction)
    #loss = torch.log(loss) + 0.001*torch.norm(net.posy.w, p=1)
    loss += 0.001*torch.norm(net.posy.w, p=1)

    loss.backward()
    optimizer.step()
    
    running_loss += loss.item()
    
    if epoch%1==0: 
        print(running_loss/1000)
        running_loss = 0
    

0.007456316947937012
0.05726411819458008
0.993149169921875
14.257560546875
1.268783447265625
0.06069847106933594
0.0019337350130081176
3.091701171875
333.97975
0.040431156158447265
0.010803759574890137
1.1575059814453126
1.7184486083984376
0.011640181541442871
0.011935436248779296
0.03442788314819336
0.000636514127254486
0.0001996240019798279
0.04415350341796875
0.14709719848632813
0.0009865223169326783
0.0334168701171875
7.36480615234375
1.945213134765625
2.089565185546875
1.0725968017578125
5.2891135215759276e-05
0.2642780151367187
1.816984130859375
3.637405029296875
2.859822998046875
72.9176953125
0.014842097282409667
0.006103195190429687
1.3842847900390625
0.024494821548461915
1.0879381103515624
0.007654178619384766
0.480587158203125
0.07872679901123047
1.103445068359375
3.43702783203125
1.3836959228515624
0.015435029983520508
0.24208116149902345
0.433352783203125
0.009395671844482421
1.2020196533203125
0.9599261474609375
0.06300180435180663
75.4420859375
0.0009559082984924316
2.00

In [70]:
loss_list

tensor([   36.175,   861.884,    52.206, 141177.516,   482.376],
       grad_fn=<CopySlices>)

In [71]:
print(loss)
print(torch.log(loss))
print(0.001*torch.norm(net.posy.w, p=1))

tensor(15.268, grad_fn=<AddBackward0>)
tensor(2.726, grad_fn=<LogBackward>)
tensor(0.020, grad_fn=<MulBackward0>)


In [72]:
list(net.parameters())

[Parameter containing:
 tensor([[ 1.803,  1.529, -0.346],
         [-2.016, -0.396, -1.471],
         [-0.033, -1.269, -0.969],
         [-1.724,  1.281, -1.963],
         [-0.967, -2.078, -2.031]], requires_grad=True)]

In [73]:
state = 5 + torch.rand(5)
state

tensor([5.658, 5.218, 5.974, 5.003, 5.833])

In [74]:
compute_actual(state)

tensor([ 4.522, 18.649])

In [75]:
net(state)

tensor([    -0.187,      0.000], grad_fn=<StackBackward>)

In [13]:
az = -6*vz/tgo - 12*z/tgo**2 + 1.6
az

NameError: name 'vz' is not defined

In [ ]:
torch.stack([ax, az])

In [ ]:
compute_actual(state)

In [ ]:
optimizer.zero_grad()


input = torch.rand(5, requires_grad = True)

with torch.no_grad():
    actual = compute_actual(input)


In [ ]:
actual

In [ ]:
net_prediction = net(input)
net_prediction

In [ ]:
loss = criterion(net_prediction, actual)
loss

In [ ]:
with torch.autograd.set_detect_anomaly(True):
    loss.backward()

In [ ]:
print('started training')
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')